# Transit-oriented bicycle network development analysis
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook follows the transit-oriented development approach of palominos2020ica and applies cardillo2006spp: Take the greedy triangulation between railway/underground stations. These will be one foundation for the bike network development.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-18  
Last modified: 2020-07-02

## Preliminaries

### Imports and magics

In [ ]:
import networkx as nx
import igraph as ig
import osmnx as ox
import numpy as np
import itertools
from matplotlib import cm
from haversine import haversine
import pandas as pd
from shapely.geometry import Point,MultiPoint,LineString,Polygon
import pandas as pd
import geopandas as gpd
import csv
import copy
import math

%matplotlib inline
import matplotlib.pyplot as plt

import watermark
%load_ext watermark

In [ ]:
%watermark -n -v -m -g -iv

## Parameters

In [ ]:
placeid = "budapest"
BWq = 0.5 # Betweenness quantile

datapath = "../data/"
outpath = "../output/"

# Graph plotting
ymirrored = False

### Functions

In [ ]:
# Graph plotting preparation functions
def my_plot_reset(G, nids = False):
    reset_plot_attributes(G)
    color_nodes(G, "red", nids)
    size_nodes(G, 8, nids)

def reset_plot_attributes(G):
    """Resets node attributes for plotting.
    All black and size 0.
    """
    G.vs["color"] = "black"
    G.vs["size"] = 0
        
def color_nodes(G, color = "blue", nids = False, use_id = True):
    """Sets the color attribute of a set of nodes nids.
    """
    if nids is False:
        nids = [v.index for v in G.vs]
        use_id = False
    if use_id:
        for nid in set(nids):
            G.vs.find(id = nid)["color"] = color
    else:
        G.vs[nids]["color"] = color

def size_nodes(G, size = 5, nids = False, use_id = True):
    """Sets the size attribute of a set of nodes nids.
    """
    if nids is False:
        nids = [v.index for v in G.vs]
        use_id = False
    if use_id:
        for nid in set(nids):
            G.vs.find(id = nid)["size"] = size
    else:
        G.vs[nids]["size"] = size

def color_edges(G, color = "blue", eids = False):
    """Sets the color attribute of a set of edge nids.
    """
    if eids is False:
        G.es["color"] = color
    else:
        G.es[eids]["color"] = color
        
def width_edges(G, width = 1, eids = False):
    """Sets the width attribute of a set of edge nids.
    """
    if eids is False:
        G.es["width"] = width
    else:
        G.es[eids]["width"] = width
        
# Other functions
def round_coordinates(G, r = 7):
    for v in G.vs:
        G.vs[v.index]["x"] = round(G.vs[v.index]["x"], r)
        G.vs[v.index]["y"] = round(G.vs[v.index]["y"], r)

def mirror_y(G):
    for v in G.vs:
        y = G.vs[v.index]["y"]
        G.vs[v.index]["y"] = -y

def dist(v1,v2):
    dist = haversine((v1['x'],v1['y']),(v2['x'],v2['y']))
    return dist


def graph_from_plc(graph):
    """This function takes a open street map object 
    and returns an igraph network with edges weighted 
    with distance between nodes
    """
    nodes, edges = ox.graph_to_gdfs(graph)
    g = ig.Graph(directed=False)
    x = nodes['x'].tolist()
    y= nodes['y'].tolist()
    ids = nodes['osmid'].tolist()
    for i in range(len(x)):
        g.add_vertex(x=x[i],y=y[i],id=ids[i])
    df1 = pd.DataFrame(data=g.vs['id'],columns={'id'})
    df1['number'] = np.arange(0,len(list(g.vs())))
    edges1 = pd.merge(df1,edges,left_on='id',right_on='u')
    edges2 = pd.merge(df1,edges1,left_on='id',right_on='v')
    edgelist = (np.stack([edges2['number_x'].tolist(),edges2['number_y'].tolist()]).T).tolist()
    g.add_edges(edgelist)
    g.simplify()
    edge_list = g.get_edgelist()
    weights=[]
    for k in range(len(edge_list)):
        d = dist(g.vs()[edge_list[k][0]],g.vs()[edge_list[k][1]])
        weights.append(d)
    g.es()['weight'] = weights
    
    #bc = g.betweenness(weights='weight')
    return g


## Tyler's code from here

def osm_to_ig(node_imported,edge_imported):
    #node,edge = ox.graph_to_gdfs(G)
    node,edge = node_imported,edge_imported
    g = ig.Graph(directed=False)

    x_coords = node['x'].tolist() 
    y_coords = node['y'].tolist()
    ids = node['osmid'].tolist()
    coords=[]

    for i in range(len(x_coords)):
        g.add_vertex(x=x_coords[i],y=y_coords[i],id=ids[i])
        coords.append((x_coords[i],y_coords[i]))

    id_dict = dict(zip(g.vs['id'],np.arange(0,g.vcount()).tolist()))
    coords_dict = dict(zip(np.arange(0,g.vcount()).tolist(),coords))


    edge_list = []
    for i in range(len(edge)):
        edge_list.append([id_dict.get(edge['u'][i]),id_dict.get(edge['v'][i])])
        
    g.add_edges(edge_list)
    g.simplify()
    new_edges=g.get_edgelist()
    
    distances_list = []
    for i in range(len(new_edges)):
        distances_list.append(haversine(coords_dict.get(new_edges[i][0]),coords_dict.get(new_edges[i][1])))

    g.es()['weight']=distances_list
    return g   

## Exploratory Data Analysis

### Load networks and POIs

In [ ]:
node_bikeable = pd.read_csv(datapath+placeid+'_bikeable_nodes.csv')
edge_bikeable = pd.read_csv(datapath+placeid+'_bikeable_edges.csv')
G_bikeable = osm_to_ig(node_bikeable, edge_bikeable)

node_carall = pd.read_csv(datapath+placeid+'_carall_nodes.csv')
edge_carall = pd.read_csv(datapath+placeid+'_carall_edges.csv')
G_carall = osm_to_ig(node_carall, edge_carall)

with open(datapath+placeid+'_poi_railwaystation_nnidscarall.csv') as f:
    nnids = [int(line.rstrip()) for line in f]

round_coordinates(G_bikeable)
round_coordinates(G_carall)

# This loop is only for plotting, executed once, to mirror all y values
if not ymirrored:
    mirror_y(G_carall)
    ymirrored = True

## Plot the starting point

In [ ]:
my_plot_reset(G_carall, nnids)

#ig.plot(G_carall, outpath + placeid + '_carall_poirailway.pdf')
#ig.plot(G_carall, outpath + placeid + '_carall_poirailway.png', bbox=(800,800))
#ig.plot(G_carall)

## Routing (shortest paths)

In [ ]:
def greedy_triangulation(GT, poipairs, betweenness_quantile = 1):
    """Greedy Triangulation (GT) of a graph GT with an empty edge set.
    Distances between pairs of nodes are given by poipairs.
    
    The GT connects pairs of nodes in ascending order of their distance provided
    that no edge crossing is introduced. It leads to a maximal connected planar
    graph, while minimizing the total length of edges considered. 
    See: cardillo2006spp
    """
    
    for poipair, poipair_distance in poipairs:
        poipair_ind = (GT.vs.find(id = poipair[0]).index, GT.vs.find(id = poipair[1]).index)
        if not new_edge_intersects(GT, (GT.vs[poipair_ind[0]]["x"], GT.vs[poipair_ind[0]]["y"], GT.vs[poipair_ind[1]]["x"], GT.vs[poipair_ind[1]]["y"])):
            GT.add_edge(poipair_ind[0], poipair_ind[1], weight = poipair_distance)
            
    # Get betweenness for prioritization
    BW = GT.edge_betweenness(False, None, "weight")
    qt = np.quantile(BW, 1-betweenness_quantile)
    sub_edges = []
    for c, e in enumerate(GT.es):
        if BW[c] >= qt: 
            sub_edges.append(c)
        GT.es[c]["bw"] = BW[c]
        GT.es[c]["width"] = math.sqrt(BW[c]+1)*0.5
    # Prune
    GT = GT.subgraph_edges(sub_edges)
    
    return GT
    

def greedy_triangulation_routing(G, pois, betweenness_quantile = 1):
    """Greedy Triangulation (GT) of a graph G's node subset pois,
    then routing to connect the GT (up to a quantile of betweenness
    betweenness_quantile).
    G is an ipgraph graph, pois is a list of node ids.
    
    The GT connects pairs of nodes in ascending order of their distance provided
    that no edge crossing is introduced. It leads to a maximal connected planar
    graph, while minimizing the total length of edges considered. 
    See: cardillo2006spp
    
    Distance here is routing distance, while edge crossing is checked on an abstract 
    level.
    """
    
    # GT is the Graph with the routing of the greedy triangulation
    GT_indices = set()
    # GT_abstract is the graph with same nodes but euclidian links to keep track of edge crossings
    pois_indices = set()
    for poi in pois:
        pois_indices.add(G.vs.find(id = poi).index)
    G_temp = copy.deepcopy(G)
    for e in G_temp.es: # delete all edges
        G_temp.es.delete(e)
    GT_abstract = G_temp.subgraph(pois_indices)
    
    poipairs = poipairs_by_distance(G, pois, True)
    GT_abstract = greedy_triangulation(GT_abstract, poipairs, betweenness_quantile)
    
    # Get node pairs we need to route, sorted by distance
    routenodepairs = {}
    for e in GT_abstract.es:
        routenodepairs[(e.source_vertex["id"], e.target_vertex["id"])] = e["weight"]
    routenodepairs = sorted(routenodepairs.items(), key = lambda x: x[1])
    
    # Do the routing
    for poipair, poipair_distance in routenodepairs:
        poipair_ind = (G.vs.find(id = poipair[0]).index, G.vs.find(id = poipair[1]).index)
        sp = set(G.get_shortest_paths(poipair_ind[0], poipair_ind[1], weights = "weight", output = "vpath")[0])
        GT_indices = GT_indices.union(sp)

    GT = G.induced_subgraph(GT_indices)
    
    return (GT, GT_abstract)
    
    
def poipairs_by_distance(G, pois, return_distances = False):
    """Calculates the (weighted) graph distances on G for a subset of nodes pois.
    Returns all pairs of poi ids in ascending order of their distance. 
    If return_distances, then distances are also returned.
    """
    
    # Get poi indices
    indices = []
    for poi in pois:
        indices.append(G_carall.vs.find(id = poi).index)
    
    # Get sequences of nodes and edges in shortest paths between all pairs of pois
    poi_nodes = []
    poi_edges = []
    for c, v in enumerate(indices):
        poi_nodes.append(G.get_shortest_paths(v, indices[c:], weights = "weight", output = "vpath"))
        poi_edges.append(G.get_shortest_paths(v, indices[c:], weights = "weight", output = "epath"))

    # Sum up weights (distances) of all paths
    poi_dist = {}
    for paths_n, paths_e in zip(poi_nodes, poi_edges):
        for path_n, path_e in zip(paths_n, paths_e):
            # Sum up distances of path segments from first to last node
            path_dist = sum([G.es[e]['weight'] for e in path_e])
            if path_dist > 0:
                poi_dist[(path_n[0],path_n[-1])] = path_dist
            
    temp = sorted(poi_dist.items(), key = lambda x: x[1])
    # Back to ids
    output = []
    for p in temp:
        output.append([(G.vs[p[0][0]]["id"], G.vs[p[0][1]]["id"]), p[1]])
    
    if return_distances:
        return output
    else:
        return [o[0] for o in output]


class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
def ccw(A,B,C):
    return (C.y-A.y) * (B.x-A.x) > (B.y-A.y) * (C.x-A.x)

def segments_intersect(A,B,C,D):
    """Check if two line segments intersect (except for colinearity)
    Returns true if line segments AB and CD intersect properly.
    Adapted from: https://stackoverflow.com/questions/3838329/how-can-i-check-if-two-segments-intersect
    """
    if (A.x == C.x and A.y == C.y) or (A.x == D.x and A.y == D.y) or (B.x == C.x and B.y == C.y) or (B.x == D.x and B.y == D.y): return False # If the segments share an endpoint they do not intersect properly
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

def new_edge_intersects(G, enew):
    """Given a graph G and a potential new edge enew,
    check if enew will intersect any old edge.
    """
    E1 = Point(enew[0], enew[1])
    E2 = Point(enew[2], enew[3])
    for e in G.es():
        O1 = Point(e.source_vertex["x"], e.source_vertex["y"])
        O2 = Point(e.target_vertex["x"], e.target_vertex["y"])
        if segments_intersect(E1, E2, O1, O2):
            return True
    return False
    

In [ ]:
(GT, GT_abstract) = greedy_triangulation_routing(G_carall, nnids, BWq)

## Plot results

### Plot the abstract Greedy Triangulation

In [ ]:
color_nodes(GT_abstract, "red", False, False)
size_nodes(GT_abstract, 4, False, False)

ig.plot(GT_abstract, outpath + placeid + '_GTabstract_' + "BWq{:.2f}".format(BWq) + '.pdf')
ig.plot(GT_abstract, outpath + placeid + '_GTabstract_' + "BWq{:.2f}".format(BWq) + '.png', bbox=(800,800))
ig.plot(GT_abstract)

### Plot just the bicycle network

In [ ]:
size_nodes(GT, 0)
width_edges(GT, 2)
color_edges(GT, "blue")

ig.plot(GT, outpath + placeid + '_GTbonly_' + "BWq{:.2f}".format(BWq) + '.pdf')
ig.plot(GT, outpath + placeid + '_GTbonly_' + "BWq{:.2f}".format(BWq) + '.png', bbox=(800,800))
ig.plot(GT)

### Plot all together

In [ ]:
my_plot_reset(G_carall, nnids)
GT_ids = [v["id"] for v in GT.vs]  
color_nodes(G_carall, "blue", GT_ids)
size_nodes(G_carall, 5, GT_ids)
color_nodes(G_carall, "red", nnids)
size_nodes(G_carall, 8, nnids)

ig.plot(G_carall, outpath + placeid + '_GTall_' + "BWq{:.2f}".format(BWq) + '.pdf')
ig.plot(G_carall, outpath + placeid + '_GTall_' + "BWq{:.2f}".format(BWq) + '.png', bbox=(800,800))
ig.plot(G_carall)